# API Integration Guide

Complete guide to integrating FinCrime-LLM API into your applications.

## 1. API Setup

In [ ]:
import requests
import json

# API Configuration
API_BASE_URL = 'http://localhost:8000'
API_KEY = 'your-api-key-here'  # If authentication is enabled

headers = {
    'Content-Type': 'application/json',
    # 'Authorization': f'Bearer {API_KEY}'  # Uncomment if auth enabled
}

print('✅ API client configured')

## 2. Health Check

In [ ]:
response = requests.get(f'{API_BASE_URL}/health')
print('Health Check:')
print(json.dumps(response.json(), indent=2))

## 3. Generate SAR

In [ ]:
sar_request = {
    'country': 'Ghana',
    'subject_name': 'John Doe',
    'institution': 'Test Bank Ltd',
    'total_amount': 100000,
    'currency': 'GHS',
    'transactions': 'Multiple cash deposits under 10,000 GHS',
    'summary': 'Suspected structuring activity'
}

response = requests.post(
    f'{API_BASE_URL}/api/v1/sar/generate',
    json=sar_request,
    headers=headers
)

if response.status_code == 200:
    result = response.json()
    print('✅ SAR Generated:')
    print(f"Report ID: {result['report_id']}")
    print(f"\nContent:\n{result['sar_content'][:300]}...")
else:
    print(f'❌ Error: {response.status_code}')
    print(response.text)

## 4. KYC Assessment

In [ ]:
kyc_request = {
    'name': 'Jane Smith',
    'customer_type': 'Individual',
    'country': 'Kenya',
    'occupation': 'Import/Export Business',
    'source_of_funds': 'Business Revenue',
    'expected_volume': '100,000 - 500,000 KES'
}

response = requests.post(
    f'{API_BASE_URL}/api/v1/kyc/assess',
    json=kyc_request,
    headers=headers
)

if response.status_code == 200:
    result = response.json()
    print('✅ KYC Assessment Complete:')
    print(f"Assessment ID: {result['assessment_id']}")
    print(f"\nContent:\n{result['kyc_content'][:300]}...")
else:
    print(f'❌ Error: {response.status_code}')

## 5. Transaction Analysis

In [ ]:
transaction_request = {
    'transactions': 'Multiple rapid in-and-out transactions',
    'description': 'High velocity, minimal balance retention'
}

response = requests.post(
    f'{API_BASE_URL}/api/v1/transaction/analyze',
    json=transaction_request,
    headers=headers
)

if response.status_code == 200:
    result = response.json()
    print('✅ Transaction Analysis Complete:')
    print(f"Analysis ID: {result['analysis_id']}")
    print(f"\nContent:\n{result['analysis_content'][:300]}...")
else:
    print(f'❌ Error: {response.status_code}')

## 6. Batch Processing

In [ ]:
# Process multiple SARs
sar_cases = [
    {'country': 'Ghana', 'subject_name': 'Case 1', 'institution': 'Bank 1', 'total_amount': 50000, 'currency': 'GHS', 'transactions': 'TX1', 'summary': 'S1'},
    {'country': 'Nigeria', 'subject_name': 'Case 2', 'institution': 'Bank 2', 'total_amount': 100000, 'currency': 'NGN', 'transactions': 'TX2', 'summary': 'S2'},
    {'country': 'Kenya', 'subject_name': 'Case 3', 'institution': 'Bank 3', 'total_amount': 75000, 'currency': 'KES', 'transactions': 'TX3', 'summary': 'S3'},
]

results = []
for case in sar_cases:
    response = requests.post(f'{API_BASE_URL}/api/v1/sar/generate', json=case, headers=headers)
    if response.status_code == 200:
        results.append(response.json())

print(f'✅ Processed {len(results)} cases')

## 7. Error Handling

In [ ]:
def safe_api_call(endpoint, data, max_retries=3):
    """Make API call with retry logic."""
    for attempt in range(max_retries):
        try:
            response = requests.post(
                f'{API_BASE_URL}{endpoint}',
                json=data,
                headers=headers,
                timeout=30
            )
            
            if response.status_code == 200:
                return response.json()
            elif response.status_code == 429:  # Rate limit
                print(f'Rate limited, waiting...')
                import time
                time.sleep(2 ** attempt)
            else:
                print(f'Error {response.status_code}: {response.text}')
                return None
                
        except requests.exceptions.RequestException as e:
            print(f'Request failed: {e}')
            if attempt < max_retries - 1:
                import time
                time.sleep(2 ** attempt)
    
    return None

# Test
test_data = {'country': 'Ghana', 'subject_name': 'Test', 'institution': 'Test Bank', 'total_amount': 1000, 'currency': 'GHS', 'transactions': 'Test', 'summary': 'Test'}
result = safe_api_call('/api/v1/sar/generate', test_data)
if result:
    print('✅ API call successful')

## 8. Python Client Class

In [ ]:
class FinCrimeLLMClient:
    """Python client for FinCrime-LLM API."""
    
    def __init__(self, base_url='http://localhost:8000', api_key=None):
        self.base_url = base_url
        self.headers = {'Content-Type': 'application/json'}
        if api_key:
            self.headers['Authorization'] = f'Bearer {api_key}'
    
    def generate_sar(self, country, subject_name, institution, total_amount, currency, transactions, summary=''):
        """Generate a SAR."""
        data = {
            'country': country,
            'subject_name': subject_name,
            'institution': institution,
            'total_amount': total_amount,
            'currency': currency,
            'transactions': transactions,
            'summary': summary
        }
        response = requests.post(f'{self.base_url}/api/v1/sar/generate', json=data, headers=self.headers)
        response.raise_for_status()
        return response.json()
    
    def assess_kyc(self, name, customer_type, country, **kwargs):
        """Perform KYC assessment."""
        data = {'name': name, 'customer_type': customer_type, 'country': country, **kwargs}
        response = requests.post(f'{self.base_url}/api/v1/kyc/assess', json=data, headers=self.headers)
        response.raise_for_status()
        return response.json()
    
    def analyze_transactions(self, transactions, description=''):
        """Analyze transactions."""
        data = {'transactions': transactions, 'description': description}
        response = requests.post(f'{self.base_url}/api/v1/transaction/analyze', json=data, headers=self.headers)
        response.raise_for_status()
        return response.json()

# Usage
client = FinCrimeLLMClient()
print('✅ Client initialized')

## 9. JavaScript Example

In [ ]:
js_example = '''
// JavaScript/Node.js Example
const axios = require('axios');

const API_BASE_URL = 'http://localhost:8000';

async function generateSAR(data) {
    try {
        const response = await axios.post(
            `${API_BASE_URL}/api/v1/sar/generate`,
            data,
            {
                headers: {'Content-Type': 'application/json'}
            }
        );
        return response.data;
    } catch (error) {
        console.error('Error:', error.response.data);
        throw error;
    }
}

// Usage
const sarData = {
    country: 'Ghana',
    subject_name: 'John Doe',
    institution: 'Test Bank',
    total_amount: 100000,
    currency: 'GHS',
    transactions: 'Multiple suspicious transactions',
    summary: 'Suspected money laundering'
};

generateSAR(sarData)
    .then(result => console.log('SAR Generated:', result.report_id))
    .catch(error => console.error('Failed:', error));
'''

print('JavaScript Example:')
print(js_example)

## 10. cURL Examples

In [ ]:
curl_examples = '''
# Generate SAR
curl -X POST http://localhost:8000/api/v1/sar/generate \\
  -H "Content-Type: application/json" \\
  -d '{
    "country": "Ghana",
    "subject_name": "John Doe",
    "institution": "Test Bank",
    "total_amount": 100000,
    "currency": "GHS",
    "transactions": "Multiple deposits",
    "summary": "Suspicious activity"
  }'

# KYC Assessment
curl -X POST http://localhost:8000/api/v1/kyc/assess \\
  -H "Content-Type: application/json" \\
  -d '{
    "name": "Jane Smith",
    "customer_type": "Individual",
    "country": "Kenya",
    "occupation": "Business Owner"
  }'

# Health Check
curl http://localhost:8000/health
'''

print('cURL Examples:')
print(curl_examples)